# 3. Subsetting CPJUMP1 controls 

In this notebook, we subset control samples from the CPJUMP1 CRISPR dataset using stratified sampling. We generate 10 different random seeds to create multiple subsets, each containing 15% of the original control data stratified by plate and well metadata. This approach ensures reproducible sampling while maintaining the distribution of controls across experimental conditions.

The subsampled datasets are saved as individual parquet files for downstream analysis and model training purposes.


In [1]:
import sys
import json
import pathlib
import polars as pl

sys.path.append("../../")
from utils.data_utils import split_meta_and_features

Load helper functions

In [2]:
def load_group_stratified_data(
    profiles: str | pathlib.Path | pl.DataFrame,
    group_columns: list[str] = ["Metadata_Plate", "Metadata_Well"],
    sample_percentage: float = 0.2,
    seed: int = 0,
) -> pl.DataFrame:
    """Memory-efficiently sample a percentage of rows from each group in a dataset.

    This function performs stratified sampling by loading only the grouping columns first
    to dtermine group memberships and sizes, then samples indices from each group, and
    finally loads the full dataset filtered to only the sampled rows. This approach
    minimizes memory usage compared to loading the entire dataset upfront.

    Parameters
    ----------
    dataset_path : str or pathlib.Path
        Path to the parquet dataset file to sample from
    group_columns : list[str], default ["Metadata_Plate", "Metadata_Well"]
        Column names to use for grouping. Sampling will be performed independently
        within each unique combination of these columns
    sample_percentage : float, default 0.2
        Fraction of rows to sample from each group (must be between 0.0 and 1.0)

    Returns
    -------
    pl.DataFrame
        Subsampled dataframe containing the sampled rows from each group,
        preserving all original columns

    Raises
    ------
    ValueError
        If sample_percentage is not between 0 and 1
    FileNotFoundError
        If dataset_path does not exist
    """
    # validate inputs
    if not 0 <= sample_percentage <= 1:
        raise ValueError("sample_percentage must be between 0 and 1")

    # convert str types to pathlib types
    if isinstance(profiles, str):
        profiles = pathlib.Path(profiles).resolve(strict=True)

    # load only the grouping columns to determine groups
    if isinstance(profiles, pl.DataFrame):
        # if a polars DataFrame is provided, use it directly
        metadata_df = profiles.select(group_columns).with_row_index("original_idx")
    else:
        metadata_df = pl.read_parquet(profiles, columns=group_columns).with_row_index(
            "original_idx"
        )

    # sample indices for each group based on the group_columns
    sampled_indices = (
        metadata_df
        # group rows by the specified columns (e.g., Plate and Well combinations)
        .group_by(group_columns)
        # for each group, randomly sample a fraction of the original row indices
        .agg(
            pl.col("original_idx")
            .sample(
                fraction=sample_percentage, seed=seed
            )  # sample specified percentage from each group
            .alias("sampled_idx")  # rename the sampled indices column
        )
        # extract only the sampled indices column, discarding group identifiers
        .select("sampled_idx")
        # convert list of indices per group into individual rows (flatten the structure)
        .explode("sampled_idx")
        # extract the sampled indices as a single column series
        .get_column("sampled_idx")
        .sort()
    )

    # load the entire dataset and filter to sampled indices
    sampled_df = (
        profiles.with_row_index("idx")
        .filter(pl.col("idx").is_in(sampled_indices.implode()))
        .drop("idx")
    )

    return sampled_df

Setting input and output paths

In [3]:
# setting data path
data_dir = pathlib.Path("../0.download-data/data").resolve(strict=True)
download_module_results_dir = pathlib.Path("../0.download-data/results").resolve(
    strict=True
)

# setting directory where all the single-cell profiles are stored
profiles_dir = (data_dir / "sc-profiles").resolve(strict=True)

exp_metadata_path = (
    profiles_dir / "cpjump1" / "CPJUMP1-experimental-metadata.csv"
).resolve(strict=True)

# Setting feature selection path
shared_features_config_path = (
    profiles_dir / "cpjump1" / "feature_selected_sc_qc_features.json"
).resolve(strict=True)

# setting cpjump1 data dir
cpjump_crispr_data_dir = (data_dir / "sc-profiles" / "cpjump1-crispr-negcon").resolve()
cpjump_crispr_data_dir.mkdir(exist_ok=True)


# setting negative control
negcon_data_dir = (profiles_dir / "cpjump1" / "negcon").resolve()
negcon_data_dir.mkdir(exist_ok=True)
poscon_data_dir = (profiles_dir / "cpjump1" / "poscon").resolve()
poscon_data_dir.mkdir(exist_ok=True)

Loading data

In [4]:
# Load experimental metadata
# selecting plates that pertains to the cpjump1 CRISPR dataset
exp_metadata = pl.read_csv(exp_metadata_path)
crispr_plate_names = (
    exp_metadata.select("Assay_Plate_Barcode").unique().to_series().to_list()
)
crispr_plate_paths = [
    (profiles_dir / "cpjump1" / f"{plate}_feature_selected_sc_qc.parquet").resolve(
        strict=True
    )
    for plate in crispr_plate_names
]
# Load shared features
with open(shared_features_config_path) as f:
    loaded_shared_features = json.load(f)

shared_features = loaded_shared_features["shared-features"]

In [5]:
control_df = []
for plate_path in crispr_plate_paths:
    # load plate data and filter to controls
    plate_controls_df = pl.read_parquet(plate_path).filter(
        pl.col("Metadata_pert_type") == "control"
    )

    # split features
    controls_meta, _ = split_meta_and_features(plate_controls_df)

    # select metadata and shared features together
    controls_df = plate_controls_df.select(controls_meta + shared_features)

    # then append to list
    control_df.append(controls_df)

# concatenate dataframes
controls_df = pl.concat(control_df)

In [6]:
negcon_df = controls_df.filter(pl.col("Metadata_control_type") == "negcon")
negcon_df

Metadata_broad_sample,Metadata_ImageNumber,Metadata_Plate,Metadata_Site,Metadata_Well,Metadata_TableNumber,Metadata_ObjectNumber_cytoplasm,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ObjectNumber_cells,Metadata_ObjectNumber,Metadata_gene,Metadata_pert_type,Metadata_control_type,Metadata_target_sequence,Metadata_negcon_control_type,__index_level_0__,Nuclei_Texture_InverseDifferenceMoment_ER_5_01_256,Cytoplasm_AreaShape_Zernike_4_2,Cytoplasm_AreaShape_Zernike_9_3,Nuclei_RadialDistribution_RadialCV_AGP_2of4,Nuclei_Correlation_Correlation_DNA_HighZBF,Cells_Texture_Correlation_HighZBF_5_00_256,Cells_AreaShape_Solidity,Nuclei_RadialDistribution_MeanFrac_HighZBF_4of4,Nuclei_AreaShape_Orientation,Nuclei_Texture_Correlation_ER_5_02_256,Cytoplasm_Correlation_Correlation_DNA_LowZBF,Cytoplasm_Texture_InfoMeas2_DNA_10_01_256,Cells_RadialDistribution_FracAtD_DNA_2of4,Cells_RadialDistribution_MeanFrac_HighZBF_1of4,Nuclei_RadialDistribution_MeanFrac_Mito_4of4,Cells_Correlation_RWC_DNA_ER,Cells_Texture_InfoMeas2_ER_3_00_256,Cells_RadialDistribution_MeanFrac_HighZBF_2of4,Cytoplasm_RadialDistribution_RadialCV_ER_3of4,Nuclei_RadialDistribution_RadialCV_HighZBF_2of4,…,Nuclei_AreaShape_MinFeretDiameter,Nuclei_AreaShape_Zernike_5_3,Nuclei_AreaShape_Zernike_4_2,Nuclei_RadialDistribution_MeanFrac_HighZBF_3of4,Cytoplasm_Granularity_1_Brightfield,Nuclei_Correlation_Correlation_ER_Mito,Nuclei_AreaShape_Zernike_6_0,Cytoplasm_AreaShape_Solidity,Nuclei_RadialDistribution_FracAtD_DNA_3of4,Nuclei_AreaShape_Zernike_8_4,Nuclei_Intensity_MassDisplacement_HighZBF,Cytoplasm_Texture_Correlation_LowZBF_3_03_256,Cells_RadialDistribution_RadialCV_Mito_1of4,Nuclei_Intensity_MassDisplacement_AGP,Nuclei_Correlation_Correlation_AGP_Mito,Nuclei_RadialDistribution_MeanFrac_HighZBF_1of4,Cells_RadialDistribution_RadialCV_DNA_2of4,Nuclei_RadialDistribution_RadialCV_RNA_3of4,Cells_AreaShape_Zernike_7_3,Nuclei_RadialDistribution_MeanFrac_ER_1of4,Cytoplasm_Correlation_Overlap_ER_RNA,Cells_Texture_Correlation_LowZBF_5_01_256,Cytoplasm_Texture_Correlation_HighZBF_3_01_256,Nuclei_Texture_Correlation_LowZBF_3_02_256,Cells_RadialDistribution_RadialCV_AGP_3of4,Nuclei_RadialDistribution_MeanFrac_AGP_1of4,Cells_RadialDistribution_MeanFrac_Brightfield_1of4,Cytoplasm_Correlation_Correlation_DNA_HighZBF,Nuclei_Intensity_MassDisplacement_DNA,Cytoplasm_RadialDistribution_MeanFrac_DNA_4of4,Cells_Correlation_Correlation_AGP_DNA,Cytoplasm_Texture_InfoMeas2_RNA_3_01_256,Cells_RadialDistribution_RadialCV_DNA_4of4,Nuclei_Correlation_Correlation_DNA_LowZBF,Cytoplasm_Texture_Correlation_LowZBF_5_00_256,Cytoplasm_RadialDistribution_RadialCV_HighZBF_2of4,Nuclei_RadialDistribution_MeanFrac_Brightfield_1of4
str,i64,str,i64,str,str,i64,f64,f64,i64,i64,str,str,str,str,str,i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""BRDN0001147100""",109,"""BR00117004""",1,"""A13""","""973645104951076534500814757387…",1,1.0,1.0,1,1,null,"""control""","""negcon""","""ACAGCGCTCTCGTGTACTAT""","""NO_SITE (5 zeros)""",8898,1.283786,-0.040981,1.539885,0.436257,0.979511,-1.085045,0.916592,-0.531047,-0.71405,1.54558,0.29688,-1.00411,0.972344,-0.545127,0.582298,-0.222069,0.092287,0.389696,-0.126638,0.432598,…,0.004871,0.072083,0.08072,0.810699,0.431524,0.325434,-0.941018,0.483423,-0.020853,-0.176041,-0.521738,0.424222,-0.127634,-0.629448,-0.753969,-0.814078,-0.382098,-1.115695,-0.282277,-0.294093,-0.588443,-0.674806,-1.396787,1.035461,-0.494509,0.540402,0.324344,-0.478843,-1.135597,0.560216,0.343086,-0.458638,-0.582636,-1.230223,-0.422533,-0.106012,0.288789
"""BRDN0001147100""",109,"""BR00117004""",1,"""A13""","""973645104951076534500814757387…",2,2.0,2.0,2,2,null,"""control""","""negcon""","""ACAGCGCTCTCGTGTACTAT""","""NO_SITE (5 zeros)""",8899,0.631209,-0.947105,-0.449049,0.367252,-0.603403,0.840774,1.695294,0.4

generating 10 seeds of randomly sampled negative controls

In [7]:
for seed_val in range(10):
    # load the dataset with group stratified sub sampling
    subsampled_df = load_group_stratified_data(
        profiles=negcon_df,
        group_columns=["Metadata_Plate", "Metadata_Well"],
        sample_percentage=0.15,
        seed=seed_val,
    )

    # save the file
    subsampled_df.write_parquet(
        negcon_data_dir / f"cpjump1_crispr_negcon_seed{seed_val}.parquet"
    )

Selecting only positive controls and saving it 

In [8]:
# write as parquet file
poscon_cp_df = controls_df.filter((pl.col("Metadata_control_type") == "poscon_cp"))
poscon_cp_df.write_parquet(poscon_data_dir / "poscon_cp_df.parquet")